In [86]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import urllib
from scipy.stats import linregress
from random import *
from api_keys import weather_api_key
from api_keys import g_key
from citipy import citipy
output_data_file = "output_data/cities.csv"

In [87]:
# Select random latitude (from -90 to +90)
city_list = pd.DataFrame()
for i in range(600):
    lat=(random()-.5)*randint(-180,180)
    lon=(random()-.5)*randint(-360,360)
    coords=pd.DataFrame([[lat,lon]], columns=['lat','lon'])
    city_list=city_list.append(coords)    
city_list=city_list.reset_index()
city_list=city_list[['lat','lon']]

In [88]:
%%capture
# Use Citipy to find the nearest city to each pair of latlongs
for row in range(0,len(city_list)):
    lat = city_list.loc[row]['lat']
    lon = city_list.loc[row]['lon']    
    city = citipy.nearest_city(lat,lon)
    city_list.set_value(row, "city", city.city_name.title())
    city_list.set_value(row, "country", city.country_code.upper())

In [89]:
# Check data look for readable
city_list.head(5)
city_list.tail(5)

,lat,lon,city,country
595,-1.892098,0.024574,Takoradi,GH
596,-10.202234,-21.817459,Georgetown,SH
597,-27.842778,-11.650714,Jamestown,SH
598,8.806647,-89.860778,Santa Cruz,CR
599,8.259172,-146.978732,Hilo,US


In [97]:
%%capture
# Add lat-longs to the dataframe
base_url = "https://maps.googleapis.com/maps/api/geocode/json"
city_list['actual_lat']=''
city_list['actual_lon']=''
city_list
lng = geo_data["results"][0]["geometry"]["location"]["lng"]
for row in range(0,len(city_list)):   
    try:    
        print(row)
            city = city_list.loc[row]['city']
            country = city_list.loc[row]['country']
            address = str(city+'+'+country)
            params = {"address": address, "key": g_key}
            response = requests.get(base_url, params=params)
            geo_data = response.json()
            lat = geo_data["results"][0]["geometry"]["location"]["lat"]
            lng = geo_data["results"][0]["geometry"]["location"]["lng"]
        except IndexError:
            city_list.set_value(row, "actual_lat", lat)
            city_list.set_value(row, "actual_lon", lng)

IndentationError: unexpected indent (<ipython-input-97-f52797eb7345>, line 10)

In [98]:
# Check the DataFrame
city_list.head(5)
city_list.tail(5)

,lat,lon,city,country,actual_lat,actual_lon
595,-1.892098,0.024574,Takoradi,GH,,
596,-10.202234,-21.817459,Georgetown,SH,,
597,-27.842778,-11.650714,Jamestown,SH,,
598,8.806647,-89.860778,Santa Cruz,CR,,
599,8.259172,-146.978732,Hilo,US,,


In [99]:
%%capture
# Write all weather data into DataFrame 
base_url = 'http://api.openweathermap.org/data/2.5/weather?'
city_list['temp']=''
city_list['clouds']=''
city_list['humidity']=''
city_list['wind_speed']=''
rows_to_drop = []
for row in range(0, len(city_list)):   
    try:
        lat = city_list.loc[row]['actual_lat']
        lon = city_list.loc[row]['actual_lon']
        query_url = base_url + 'units=imperial&lat='+str(lat) + '&lon='+ str(lon) +'&appid=' + weather_api_key
        response = requests.get(query_url)
        result=response.json()
        city=city_list.loc[row]['city']
        temp=float(result['main']['temp'])
        clouds=result['clouds']['all']
        humidity=result['main']['humidity']
        wind_speed=result['wind']['speed']
        city_list.set_value(row, "temp", temp)
        city_list.set_value(row, "clouds", clouds)    
        city_list.set_value(row, "humidity", humidity)
        city_list.set_value(row, "wind_speed", wind_speed) 
    except KeyError:
        rows_to_drop.append(row)

In [100]:
city_list.head()
city_list.tail()

,lat,lon,city,country,actual_lat,actual_lon,temp,clouds,humidity,wind_speed
595,-1.892098,0.024574,Takoradi,GH,,,,,,
596,-10.202234,-21.817459,Georgetown,SH,,,,,,
597,-27.842778,-11.650714,Jamestown,SH,,,,,,
598,8.806647,-89.860778,Santa Cruz,CR,,,,,,
599,8.259172,-146.978732,Hilo,US,,,,,,


In [103]:
# Write csv file
city_list.head()
city_list.to_csv('weather_data_output.csv')

#### Latitude vs. Temperature Plot

In [118]:
%%capture
# Temperature vs. Latitude plot
x_axis = city_list['actual_lat']
y_axis = city_list['temp']
x_min=x_axis.min()-10
x_max=x_axis.max()+10
y_min=y_axis.min()-10
y_max=y_axis.max()+10
plt.title("Temperature vs. Latitude")
plt.xlabel("Latitude")
plt.ylabel("Temperture")
plt.scatter(x_axis, y_axis, marker="o", color="red")
plt.savefig('Output/Fig1_Temp_Latitude.png')
plt.show()

#### Latitude vs. Humidity Plot

In [119]:
%%capture
# Humidity (%) vs. Latitude
x_axis = city_list['actual_lat']
y_axis = city_list['humidity']
x_min=x_axis.min()-10
x_max=x_axis.max()+10
y_min=y_axis.min()-10
y_max=y_axis.max()+10
plt.xlim(x_min,x_max)
plt.ylim(y_min,y_max)
plt.title("Humidity (%) vs. Latitude")
plt.xlabel("Latitude")
plt.ylabel("Humidity (%)")
plt.scatter(x_axis, y_axis, marker="o", color="red")
plt.savefig('Output/Fig2_Humidity_Latitude.png')
plt.show()

#### Latitude vs. Cloudiness Plot

In [120]:
%%capture
# Cloudiness (%) vs. Latitude
x_axis = city_list['actual_lat']
y_axis = city_list['clouds']
x_min=x_axis.min()-10
x_max=x_axis.max()+10
y_min=y_axis.min()-10
y_max=y_axis.max()+10
plt.xlim(x_min,x_max)
plt.ylim(y_min,y_max)
plt.title("Cloudiness (%) vs. Latitude")
plt.xlabel("Latitude")
plt.ylabel("Cloudiness (%)")
plt.scatter(x_axis, y_axis, marker="o", color="red")
plt.savefig('Output/Fig3_Cloudiness_Latitude.png')
plt.show()

#### Latitude vs. Wind Speed Plot

In [122]:
%%capture
# Wind Speed (mph) vs. Latitude
x_axis = city_list['actual_lat']
y_axis = city_list['wind_speed']
x_min=x_axis.min()-10
x_max=x_axis.max()+10
y_min=y_axis.min()-10
y_max=y_axis.max()+10
plt.xlim(x_min,x_max)
plt.ylim(y_min,y_max)
plt.title("Wind Speed (mph) vs. Latitude")
plt.xlabel("Latitude")
plt.ylabel("Wind Speed (mph)")
plt.scatter(x_axis, y_axis, marker="o", color="red")
plt.savefig('Output/Fig4_WindSpeed_Latitude.png')
plt.show()